1. Для 1 любого видео восстановить траекторию движения (t-вектор). Выполнить визуализацию. Определить параметры которые влияют на "точность" определения вектора t. 
2. Использовать решение на базе нейронных сетей. Любые идеи. 
3. ***slam прикрутить. 

Алгоритм:
1. Считываем по кадру из видео, каждый кадр преобразуем в черно-белый.
2. Получаем особые точки и дескрипторы точек кадра.
3. Сопоставляем точки с двух соседних кадров по их дискрипторам.
4. Получаем оценочную матрицу по точкам двух кадров.
5. Сохраняем матрицы поворота и смещения камеры, позиции камеры на каждом кадре.
6. Ключевые точки и дескрипторы текущего кадра сохраняем для дальнейших вычислений на следующей итерации.
7. Визуализирум траекторию положений камеры и ее направления в каждой точке.
8. Сохраняем массивы с матрицами углов и смещений в npz-файл, т к очередное получение данных матриц занимает много времени.

In [1]:
import plotly.graph_objects as gr
import numpy as np
import cv2
import os

from tqdm import tqdm

# function for vizualize cemera trajectory and direction
def visualize_trajectory(rotation, positions, title='Camera motion'):
    fig = gr.Figure()

    # add positions trace
    fig.add_trace(gr.Scatter3d(x=positions[:, 0], y=positions[:, 1], z=positions[:, 2],
                               marker=dict(size=1.2, color='purple')))

    # add camera orientation traces
    for (p, r) in zip(positions, rotation):
        point2 = p + 0.5 * r[:, 2]
        fig.add_trace(gr.Scatter3d(x=[p[0], point2[0]], y=[p[1], point2[1]], z=[p[2], point2[2]],
                                   mode='lines', line=dict(width=2, color='red')))

    fig.update_layout(title=title, showlegend=False)
    fig.show()


In [3]:
# match features between images keypoints and filter matching points by distance
def match_and_filtering(descriptor1, descriptor2, matcher, threshold: float):
    filtered_matches = list()

    # k best matches between descriptors
    matches = matcher.knnMatch(descriptor1, descriptor2, k=2)

    for m, n in matches:
        if m.distance * 1. / n.distance <= threshold:
            filtered_matches.append(m)

    return filtered_matches


In [ ]:
# function for create camera trajectory by video
def process_video(source_video: str, method=cv2.LMEDS, threshold: float=1.):
    cap = cv2.VideoCapture(source_video)

    # create SIFT detector and BFMatcher objects for all video frames
    sift_detector = cv2.SIFT_create()
    matcher = cv2.BFMatcher_create(cv2.NORM_L2, crossCheck=False)

    # keypoints and descriptors from prev frame
    kps1, des1 = None, None

    # create_array with points of camera trajectory
    trajectory = np.array([[0, 0, 0]])

    # create general list with rotations matrix corresponds to each video frame
    rotations_list = [np.zeros((3, 3))]
    
    # camera positions
    positions = [np.array([0, 0, 0])]

    cam_matrix = np.eye(4)
    T = np.eye(4)

    # camera matrix
    K = np.array([[3000, 0 , cap.get(cv2.CAP_PROP_FRAME_WIDTH) / 2], [0, 3000, cap.get(cv2.CAP_PROP_FRAME_HEIGHT) / 2], [0, 0, 1]])

    while True:
        is_success, frame = cap.read()

        if not is_success:
            break

        # convert frame to gray
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # detect frame keypoints
        kps2, des2 = sift_detector.detectAndCompute(frame_gray, None)

        if kps1 is not None:
            # match keypoints from 2 frames, filtering by distance
            matches = match_and_filtering(des1, des2, matcher, 0.3)

            points1 = np.array([kps1[m.queryIdx].pt for m in matches])
            points2 = np.array([kps2[m.trainIdx].pt for m in matches])

            # calculate essential matrix to match camera positons between 2 frames
            e_mat, mask = cv2.findEssentialMat(points1, points2, K, method=method, threshold=threshold)
            _, R, t, _ = cv2.recoverPose(e_mat, points1, points2, K, mask=mask)

            T[:3, :3] = R
            T[:3, 3] = t.T

            cam_matrix = np.dot(cam_matrix, T)
            trajectory = np.vstack([trajectory, cam_matrix[:3, 3]])
            rotations_list.append(cam_matrix[:3, :3])
            
            positions.append(positions[-1] + np.dot(R, t).T[0])
            
        # save keypoints and descriptors from current frame to use in next calculations
        kps1 = kps2
        des1 = des2

    cap.release()

    return rotations_list, trajectory, np.array(positions)


In [5]:
estimate_methods = {cv2.LMEDS: 'LMEDS', cv2.RANSAC: 'RANSAC'}

In [ ]:
# process single video
def video_processor(video_path: str, method: int, threshold: float):
    global data_folder, estimate_methods

    if method == cv2.LMEDS:
        threshold = 1.

    # get camera rotations and trajectory
    rotations_list, trajectory, positions = process_video(video_path, method, threshold)

    # visualize camera motion, get fps value from path
    start = video_path.rfind('_')
    end = video_path.find('fps')
    title = f'Camera motion, {estimate_methods[method]}-{threshold}, video fps = {video_path[start+1:end]}'
    visualize_trajectory(rotations_list, trajectory, title)

    # save camera rotations and trajectory to npz-file
    npz_filename = os.path.join(data_folder, f'data_{estimate_methods[method]}-{threshold}_{video_path[start+1:end]}fps.npz')
    np.savez(npz_filename, R=rotations_list, trajectory=trajectory, positions=positions)


Используем видео с различными значением fps (fps = 5, 10, 20), чтобы подобрать видео с достаточным смещением пикселей между двумя кадрами. Также проверяем два метода для вычисления оценочной матрицы (cv2.LMEDS, cv2.RANSAC), и пороговые значения от 1 до 3ех для метода cv2.RANSAC. По графикам, полученным для различных комбинаций fps и методов, видно, что ближайшей является траектория при fps == 10, методом вычисления матрицы cv2.RANSAC с пороговым значением 2.0. В дальнейшем используем данное видео: <a href="https://drive.google.com/file/d/1jcTTa6b_1FG8GAi6ChaRziC0N0UpmVvZ/view?usp=sharing">ссылка на видео</a>

In [ ]:
# folder that contains videos with fps = 5, 10, 20
videos_folder = '/media/vik/SamsungSSD7/Courses/peleng-cources/HW_10/videos'

# folder for saving npz-files with trajectory and rotations arrays
data_folder = '/media/vik/SamsungSSD7/Courses/peleng-cources/HW_10/saved_data'

if not os.path.exists(data_folder):
    os.mkdir(data_folder)

# method's thresholds for find estimate matrix
methods_ths = {cv2.LMEDS: [1.], cv2.RANSAC: [1., 2., 3.]}

# iterate through videos with different fps and caclculate they trajectories
for video_path in tqdm(os.listdir(videos_folder)):
    print(video_path)

    for method, thresholds in methods_ths.items():
        for th in thresholds:
            video_processor(os.path.join(videos_folder, video_path), method=method, threshold=th)
    

In [4]:
npz_filename = '/media/vik/SamsungSSD7/Courses/peleng-cources/HW_10/saved_data/data_RANSAC-2.0_10fps.npz'

# load arrays from npz-file
data = np.load(npz_filename)
trajectory = data['trajectory']
rotations_list = data['R']
positions = data['positions']

visualize_trajectory(rotations_list, positions)


На точность построения траектории оказывают влияние пороговые значения для фильтрации точек после их сопоставления на соседних кадрах, смещение между соседними кадрами (различное fps), выбор метода для вычисления оценочной матрицы (cv2.LMEDS, cv2.RANSAC) в комбинации с порогами для данных методов (от 1 до 3ех для метода cv2.RANSAC), разрешение кадров видео. Точность построения траектории при использовании модели LoFTR оказалась хуже точности SIFT для данного видео. Ее работа требует больших ресурсов по памяти, поэтому нет возможности подать на вход картинку в полном разрешении, также на CUDA модель работает медленнее, чем SIFT. Ниже приведены траектории, построенные с использованием SIFT и LoFTR.

In [ ]:
npz_filename = '/media/vik/SamsungSSD7/Courses/peleng-cources/HW_10/saved_data/data_LoFTR_LMEDS_10fps.npz'

# load arrays from npz-file
data = np.load(npz_filename)
trajectory = data['trajectory']
rotations_list = data['R']
positions = data['positions']

visualize_trajectory(rotations_list, positions)


![img](./plots/plot_RANSAC_10pfs.png)
![img](./plots/plot_LoFTR_10fps.png)